<a href="https://colab.research.google.com/github/moeedkh/Typescript-Portfolio/blob/main/Stocks_Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded = files.upload()


Saving stocks.csv to stocks.csv


In [5]:
import pandas as pd
import numpy as np
from ta.trend import ADXIndicator, MACD, SMAIndicator
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

# Load the dataset
df = pd.read_csv('stocks.csv', encoding='latin1', low_memory=False)

# Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Convert numeric columns to appropriate data types and handle any issues
numeric_cols = ['Open', 'High', 'Low', 'Close', 'Change', 'Volume']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Sort DataFrame by date and symbol
df = df.sort_values(['Symbol', 'Date'])

# Define calculation windows
adx_window = 14
macd_slow = 26
macd_fast = 12
macd_signal = 9
rsi_window = 14
bb_window = 20
bb_std_dev = 2
obv_sma_window = 200

# Prepare an empty DataFrame to store the results
results_df = pd.DataFrame()

# Loop through each symbol
symbols = df['Symbol'].unique()

for symbol in symbols:
    # Filter data for the current symbol and create a copy
    symbol_data = df[df['Symbol'] == symbol].copy()

    # Ensure sufficient data points for all calculations
    if len(symbol_data) < max(adx_window, macd_slow, 200, rsi_window, bb_window, obv_sma_window):
        print(f"Skipping symbol {symbol}: insufficient data points ({len(symbol_data)})")
        continue

    try:
        # ADX calculation
        adx_indicator = ADXIndicator(high=symbol_data['High'], low=symbol_data['Low'], close=symbol_data['Close'], window=adx_window)
        symbol_data['ADX'] = adx_indicator.adx()
        symbol_data['+DI'] = adx_indicator.adx_pos()
        symbol_data['-DI'] = adx_indicator.adx_neg()
        symbol_data['ADX Interpretation'] = symbol_data['ADX'].apply(lambda x: 'Strong Trend' if x > 20 else 'Weak/No Trend')
        symbol_data['DI Trend'] = symbol_data.apply(
            lambda row: 'Bullish' if row['+DI'] > row['-DI'] else ('Bearish' if row['-DI'] > row['+DI'] else 'Neutral'),
            axis=1
        )

        # MACD calculation
        macd_indicator = MACD(close=symbol_data['Close'], window_slow=macd_slow, window_fast=macd_fast, window_sign=macd_signal)
        symbol_data['MACD'] = macd_indicator.macd()
        symbol_data['MACD Signal Line'] = macd_indicator.macd_signal()
        symbol_data['MACD Histogram'] = macd_indicator.macd_diff()
        symbol_data['MACD Interpretation'] = 'Hold'

        # RSI calculation
        rsi_indicator = RSIIndicator(close=symbol_data['Close'], window=rsi_window)
        symbol_data['RSI'] = rsi_indicator.rsi()
        symbol_data['RSI Interpretation'] = 'None'
        if len(symbol_data) >= 7:
            rsi_last_7 = symbol_data['RSI'].tail(7)
            if rsi_last_7.is_monotonic_increasing:
                symbol_data.loc[symbol_data.index[-1], 'RSI Interpretation'] = 'Rising'
            elif rsi_last_7.is_monotonic_decreasing:
                symbol_data.loc[symbol_data.index[-1], 'RSI Interpretation'] = 'Falling'
            else:
                symbol_data.loc[symbol_data.index[-1], 'RSI Interpretation'] = 'Horizontal'

        # Bollinger Bands calculation
        bb_indicator = BollingerBands(close=symbol_data['Close'], window=bb_window, window_dev=bb_std_dev)
        symbol_data['BB_MiddleBand'] = bb_indicator.bollinger_mavg()
        symbol_data['BB_UpperBand'] = bb_indicator.bollinger_hband()
        symbol_data['BB_LowerBand'] = bb_indicator.bollinger_lband()
        symbol_data['BB Interpretation'] = 'Hold'
        buy_mask = symbol_data['Close'] < symbol_data['BB_LowerBand']
        symbol_data.loc[buy_mask, 'BB Interpretation'] = 'Buy (Oversold)'
        sell_mask = symbol_data['Close'] > symbol_data['BB_UpperBand']
        symbol_data.loc[sell_mask, 'BB Interpretation'] = 'Sell (Overbought)'

        # OBV calculation
        symbol_data['OBV'] = (symbol_data['Volume'] * (symbol_data['Close'] - symbol_data['Close'].shift(1)).apply(
            lambda x: 1 if x > 0 else (-1 if x < 0 else 0))).fillna(0).cumsum()
        symbol_data['200 SMA OBV'] = symbol_data['OBV'].rolling(window=obv_sma_window).mean()
        symbol_data['OBV Interpretation'] = 'None'
        bullish_obv_mask = symbol_data['OBV'] > symbol_data['200 SMA OBV']
        symbol_data.loc[bullish_obv_mask, 'OBV Interpretation'] = 'Bullish'
        bearish_obv_mask = symbol_data['OBV'] < symbol_data['200 SMA OBV']
        symbol_data.loc[bearish_obv_mask, 'OBV Interpretation'] = 'Bearish'

        # SMA calculation for Golden Cross / Death Cross
        sma_50 = SMAIndicator(close=symbol_data['Close'], window=50)
        sma_200 = SMAIndicator(close=symbol_data['Close'], window=200)
        symbol_data['50 SMA'] = sma_50.sma_indicator()
        symbol_data['200 SMA'] = sma_200.sma_indicator()
        symbol_data['SMA Interpretation'] = 'None'
        golden_cross_mask = (symbol_data['50 SMA'] > symbol_data['200 SMA']) & (
                symbol_data['50 SMA'].shift(1) <= symbol_data['200 SMA'].shift(1))
        symbol_data.loc[golden_cross_mask, 'SMA Interpretation'] = 'Golden Cross'
        death_cross_mask = (symbol_data['50 SMA'] < symbol_data['200 SMA']) & (
                symbol_data['50 SMA'].shift(1) >= symbol_data['200 SMA'].shift(1))
        symbol_data.loc[death_cross_mask, 'SMA Interpretation'] = 'Death Cross'
        ongoing_golden_cross_mask = (symbol_data['50 SMA'] > symbol_data['200 SMA']) & (
                symbol_data['SMA Interpretation'] == 'None')
        symbol_data.loc[ongoing_golden_cross_mask, 'SMA Interpretation'] = 'Ongoing Golden Cross'
        ongoing_death_cross_mask = (symbol_data['50 SMA'] < symbol_data['200 SMA']) & (
                symbol_data['SMA Interpretation'] == 'None')
        symbol_data.loc[ongoing_death_cross_mask, 'SMA Interpretation'] = 'Ongoing Death Cross'

        # Filter only the most recent day
        max_date = symbol_data['Date'].max()
        symbol_last_day = symbol_data[symbol_data['Date'] == max_date]

        # Append results
        results_df = pd.concat([results_df, symbol_last_day])

    except Exception as e:
        print(f"Error processing symbol {symbol}: {e}")
        continue

# Filter the final results to the last available date
if not results_df.empty:
    most_recent_date = results_df['Date'].max()
    results_df = results_df[results_df['Date'] == most_recent_date]

    # Select relevant columns
    results_df = results_df[['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume',
                             'ADX', '+DI', '-DI', 'ADX Interpretation', 'DI Trend',
                             'MACD', 'MACD Signal Line', 'MACD Histogram', 'MACD Interpretation',
                             'RSI', 'RSI Interpretation', 'BB_MiddleBand', 'BB_UpperBand', 'BB_LowerBand',
                             'BB Interpretation', 'OBV', '200 SMA OBV', 'OBV Interpretation',
                             '50 SMA', '200 SMA', 'SMA Interpretation']]

    # Sort results by volume
    results_df = results_df.sort_values(by='Volume', ascending=False)

    # Save to Excel
    output_file_path = 'C:\\Users\\abdul\\ADX_MACD_RSI_BB_OBV_SMA_Latest_Analysis.xlsx'
    results_df.to_excel(output_file_path, index=False)

    print(f"\nFinal DataFrame with ADX, MACD, RSI, Bollinger Bands, OBV, and SMA analysis for the most recent date:")
    print(results_df)
    print(f"\nData has been exported to {output_file_path}")
else:
    print("\nNo analysis results found.")

Skipping symbol AATM: insufficient data points (143)
Skipping symbol ACIETF: insufficient data points (149)
Skipping symbol AEL: insufficient data points (28)
Skipping symbol AGHA-APR: insufficient data points (21)
Skipping symbol AGHA-AUG: insufficient data points (26)
Skipping symbol AGHA-DEC: insufficient data points (50)
Skipping symbol AGHA-FEB: insufficient data points (17)
Skipping symbol AGHA-JAN: insufficient data points (38)
Skipping symbol AGHA-JUL: insufficient data points (35)
Skipping symbol AGHA-JUN: insufficient data points (20)
Skipping symbol AGHA-MAR: insufficient data points (27)
Skipping symbol AGHA-MAY: insufficient data points (40)
Skipping symbol AGHA-NOV: insufficient data points (35)
Skipping symbol AGHA-OCT: insufficient data points (25)
Skipping symbol AGHA-SEP: insufficient data points (42)
Skipping symbol AGL-APR: insufficient data points (21)
Skipping symbol AGL-AUG: insufficient data points (29)
Skipping symbol AGL-DEC: insufficient data points (26)
Skip